In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Oct 11 15:36:48 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A40                     On  |   00000000:CE:00.0 Off |                    0 |
|  0%   37C    P0             76W /  300W |   25853MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%%capture
!pip install datasets transformers huggingface_hub torchaudio librosa jiwer
!apt install git-lfs
!pip install accelerate -U
!pip install nvidia-smi
!pip install transformers[torch]
!pip install wandb
!pip install matplotlib seaborn
!pip install pyctcdecode
!pip install kenlm
!pip install evaluate
import torch
import random
import pandas as pd
from datasets import load_dataset, Audio
from IPython.display import display, HTML, Audio as IPythonAudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Trainer, TrainingArguments
import re
import json
from collections import Counter
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import wandb
import os
from typing import Any, Dict, List, Union
from dataclasses import dataclass
from huggingface_hub import HfFolder
from transformers import Wav2Vec2ProcessorWithLM, Wav2Vec2BertForCTC
import torch
from transformers.pipelines import pipeline
from datasets import load_dataset
import pandas as pd
import evaluate
import random

In [ ]:
from transformers import pipeline, Wav2Vec2ProcessorWithLM, AutoFeatureExtractor, Wav2Vec2BertForCTC, Wav2Vec2BertProcessor
from transformers.pipelines.pt_utils import KeyDataset
from tqdm import tqdm
import torch
import unicodedata
import re
token = os.getenv("HF_TOKEN")  # Replaced secret with secure environment variableCGFhnfQzLEzVmfLlIZhDwYuXAgomjoTtmQ"
HfFolder.save_token(token)

In [ ]:
from datasets import load_dataset, DatasetDict

test5hr = load_dataset("Beijuka/Bemba_test_5hr")['test']
test5hr = test5hr.rename_column('sentence', 'transcription')
print(test5hr)

Dataset({
    features: ['audio', 'transcription', 'duration', 'source'],
    num_rows: 2351
})


In [ ]:
import unicodedata
import re
def preprocess_text(batch):
    batch['transcription'] = [text.lower() for text in batch['transcription']]
    batch['transcription'] = [unicodedata.normalize('NFKC', text) for text in batch['transcription']]
    batch['transcription'] = [re.sub(r'[\’\ʻ\ʼ\ʽ\‘\']', '', text) for text in batch['transcription']]
    batch['transcription'] = [re.sub(r'[^\w\s\']', '', text) for text in batch['transcription']]
    batch['transcription'] = [re.sub(r'_', ' ', text) for text in batch['transcription']]
    batch['transcription'] = [' '.join(text.split()) for text in batch['transcription']]
    return batch

test5hr = test5hr.map(preprocess_text, batched=True)

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["transcription"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}
vocab_test = test5hr.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test5hr.column_names)
vocab_list = list(set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

Map:   0%|          | 0/2351 [00:00<?, ? examples/s]

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'y': 24,
 'z': 25}

In [ ]:
model_ids = [
    "asr-africa/w2v-bert-2.0_BIG-C_corpus_Bemba_1hr_v1",
    "asr-africa/W2V2_Bert_BIG-C_BEMBA_5hr_v1",
    "asr-africa/W2V2_Bert_BIG-C_BEMBA_10hr_v1",
    "asr-africa/W2V2_Bert_BIG_C_Bemba_20hr_v1",
    "asr-africa/W2V2_Bert_BIG_C_Bemba_50hr_v1",
    "asr-africa/W2V2_Bert_BIG_C_Bemba_100hr_v1"
]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")
device

device(type='cuda')

In [ ]:

for model_id in model_ids:
    original_test5hr = load_dataset("Beijuka/Bemba_test_5hr")['test']
    original_test5hr = original_test5hr.rename_column('sentence', 'transcription')
    test5hr = original_test5hr.map(preprocess_text, batched=True)
    processor = Wav2Vec2ProcessorWithLM.from_pretrained(model_id)
    model = Wav2Vec2BertForCTC.from_pretrained(model_id)
    model.to(device)

    asr = pipeline("automatic-speech-recognition", model=model, device=device,
                   tokenizer=processor.tokenizer, feature_extractor=processor.feature_extractor,
                   decoder=processor.decoder)

    predictions = [asr(audio)['text'] for audio in tqdm(KeyDataset(test5hr, "audio"),
                                                       total=len(test5hr),
                                                       desc=f"Processing with {model_id}")]
    wer_scores = [wer_metric.compute(predictions=[pred], references=[ref]) for pred, ref in zip(predictions, test5hr['transcription'])]
    cer_scores = [cer_metric.compute(predictions=[pred], references=[ref]) for pred, ref in zip(predictions, test5hr['transcription'])]

    results = {
        "Prediction": predictions,
        "Reference": test5hr['transcription'],
        "WER": wer_scores,
        "CER": cer_scores,
        "Source": test5hr['source']
    }

    df = pd.DataFrame(results)
    overall_wer = wer_metric.compute(predictions=df["Prediction"], references=df["Reference"])
    overall_cer = cer_metric.compute(predictions=df["Prediction"], references=df["Reference"])

    print("Calculated overall WER:", overall_wer)
    print("Calculated overall CER:", overall_cer)

    average_metrics_by_source = df.groupby("Source").agg({"WER": "mean", "CER": "mean"}).reset_index()
    print("\nAverage WER and CER by Source:")
    print(average_metrics_by_source)

    csv_filename = f'{model_id.replace("/", "_")}.csv'
    df.to_csv(csv_filename, index=False)
    print(f"Results saved to {csv_filename}")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

alphabet.json:   0%|          | 0.00/193 [00:00<?, ?B/s]

language_model/attrs.json:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

language_model/unigrams.txt:   0%|          | 0.00/970k [00:00<?, ?B/s]

5gram.bin:   0%|          | 0.00/67.3M [00:00<?, ?B/s]

...

language_model/unigrams.txt:   0%|          | 0.00/970k [00:00<?, ?B/s]

5gram.bin:   0%|          | 0.00/67.3M [00:00<?, ?B/s]

alphabet.json:   0%|          | 0.00/193 [00:00<?, ?B/s]

Processing with asr-africa/W2V2_Bert_BIG_C_Bemba_100hr_v1: 100%|██████████| 2351/2351 [07:18<00:00,  5.36it/s]


Calculated overall WER: 0.4157732224032936
Calculated overall CER: 0.08959081029166456

Average WER and CER by Source:
           Source       WER       CER
0       BigC_test  0.546619  0.133548
1  csikasote_test  0.359468  0.069925
Results saved to asr-africa_W2V2_Bert_BIG_C_Bemba_100hr_v1.csv
